In [1]:
import requests
from selenium import webdriver
from PIL import Image
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ChromeOptions
from selenium.webdriver import ActionChains
import pandas as pd
import time
import random

In [2]:
chrome_options=Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
option = ChromeOptions()
option.add_experimental_option('excludeSwitches',['enable-automation'])

# Driver打开豆瓣

In [3]:
url = "https://www.douban.com/"
driver = webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe",)# chrome_options=chrome_options, options=option)
driver.get(url)

# 登录

## 滑动验证

In [4]:
def get_img():
    time.sleep(5)
    #切换到iframe('tcaptcha_iframe')
    driver.switch_to.frame('tcaptcha_iframe')
    
    #获取缺货、完整图片的url
    src = driver.find_element_by_xpath('//*[@class="tc-bg"]/img').get_attribute('src')
    src_background = src.replace('img_index=1', 'img_index=0')
    
    #下载缺失的图片和完整的图片
    open('./img_canque.png', 'wb').write(requests.get(url=src).content)
    open('./img_wanzheng.png', 'wb').write(requests.get(url=src_background).content)
    print("下载完成！")
    
    #加载缺失、完整的图片
    captcha_queshi = Image.open('./img_canque.png')
    captcha_wanzheng = Image.open('./img_wanzheng.png')
    
    return captcha_queshi, captcha_wanzheng

In [5]:
def resize_img(img):#将图片还原成原尺寸
    #原图片为680×390px 渲染的大小为283×162px
    a = 2.403
    (x, y) = img.size
    x_resize = int(x // a)
    y_resize = int(y // a)
    img = img.resize((x_resize, y_resize), Image.ANTIALIAS)
    
    return img

In [6]:
def is_pixel_equal(img1, img2, x, y):#比较两张图片同一个点的像数值，差距大于设置的标准返回False
    pixel1, pixel2 = img1.load()[x, y], img2.load()[x, y]
    sub_index = 100 #设置的差距
    
    #比较RGB分量的值
    if abs(pixel1[0] - pixel2[0]) < sub_index and abs(pixel1[1] - pixel2[1]) < sub_index and abs(pixel1[2] - pixel2[2]) < sub_index:
        return True
    else:
        return False

In [7]:
def get_gap_offest(img1, img2):#获得缺失的位置
    distance = 70
    
    for i in range(distance, img1.size[0]):
        for j in range(img1.size[1]):
            #两张图片对比,(i,j)像素点的RGB差距，过大则为偏移值
            if not is_pixel_equal(img1, img2, i, j):
                distance = i
                return distance
    return distance

In [8]:
def get_track(distance):#计算滑块的移动轨迹
    distance -= 30
    a = distance / 4
    track = [a, a, a, a]
    return track

In [9]:
def shake_mouse():#模拟人手释放鼠标抖动
    ActionChains(driver).move_by_offset(xoffset=-2, yoffset=0).perform()
    ActionChains(driver).move_by_offset(xoffset=2, yoffset=0).perform()

In [10]:
def operate_slider(track):#拖动滑块
    slider_bt = driver.find_element_by_xpath('//*[@class="tc-drag-thumb"]')#定位滑块
    ActionChains(driver).click_and_hold(slider_bt).perform()#点击拖动滑块
    
    for i in track:
        ActionChains(driver).move_by_offset(xoffset=i, yoffset=0).perform()
        time.sleep(random.random() / 100)
    time.sleep(random.random())
    
#     back_tracks = [-1, -0.5, -1]
#     for i in back_tracks:
#         time.sleep(random.random() / 100)
#         ActionChains(driver).move_by_offset(xoffset=i, yoffset=0).perform()
        
    shake_mouse()
    time.sleep(random.random())
    ActionChains(driver).release().perform()#松开滑块
    

## 短信登录

In [11]:
def tel_login():
    #切换到iframe[0]标签
    driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[0])
    
    #点击并输入手机号
    driver.find_element_by_xpath('//*[@name="phone"]').click()
    driver.find_element_by_xpath('//*[@name="phone"]').send_keys("18939200916")
    
    #点击获取验证码
    driver.find_element_by_xpath('//*[@class="account-form-field-code"]').click()
    driver.find_element_by_id('code').click()
    
    #滑动验证
    img1, img2 = get_img()
    img1 = resize_img(img1)
    img2 = resize_img(img2)
    distance = get_gap_offest(img1, img2)
    track = get_track(distance)
    operate_slider(track)
    time.sleep(5)
    
    #点击并输入验证码
    driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[0])
    driver.find_element_by_xpath('//*[@class="account-form-field account-form-codes"]/input').click()
    verification_code = input('请输入验证码：')
    driver.find_element_by_xpath('//*[@class="account-form-field account-form-codes"]/input').send_keys(verification_code)
    
    #点击 登陆豆瓣
    driver.find_element_by_xpath('//*[@class="account-form-field-submit "]').click()

In [12]:
tel_login()

下载完成！
请输入验证码：1566


## 密码登录

> 密码登录在完成滑动验证后需要扫描二维码

In [ ]:
def user_login():#登录
    input_username_password('18939200916', 'z20011225')#账号和密码
    
    img1, img2 = get_img()
    img1 = resize_img(img1)
    img2 = resize_img(img2)
    distance = get_gap_offest(img1, img2)
    track = get_track(distance)
    operate_slider(track)

In [5]:
def input_username_password(username, password):#输入账号密码登录
    #切换到iframe[0]标签
    driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[0])
    
    #切换到账号密码登录
    driver.find_element_by_class_name("account-tab-account").click()
    
    #输入账号
    driver.find_element_by_id("username").click()
    driver.find_element_by_id("username").send_keys(username)
    
    #输入密码
    driver.find_element_by_id("password").click()
    driver.find_element_by_id("password").send_keys(password)
    
    #点击 登录豆瓣
    driver.find_element_by_class_name("account-form-field-submit").click()

In [ ]:
user_login()

# 获取电影信息

In [17]:
def movie():#获取电影的基本信息
    driver.find_element_by_xpath('//*[@class="global-nav-items"]//li[3]').click()#点击电影
    
    windows = driver.window_handles
    driver.switch_to.window(windows[-1])#切换到新打开的标签
    
    driver.find_element_by_xpath('//*[@class="nav-items"]//li[5]').click()#点击排行榜
    time.sleep(5)
    
    movie_name = driver.find_elements_by_xpath('//*[@class="pl2"]/a')#名字
    movie_info = driver.find_elements_by_xpath('//p[@class="pl"]')#基本信息
    movie_score = driver.find_elements_by_xpath('//*[@class="rating_nums"]')#分数
    movie_comment = driver.find_elements_by_xpath('//span[@class="pl"]')#评价人数
    name_list, info_list, score_list, comment_list = [], [], [], []
    for i in range(len(movie_name)):
        name_list.append(movie_name[i].text)
        info_list.append(movie_info[i].text)
        score_list.append(movie_score[i].text)
        comment_list.append(movie_comment[i].text)
    return name_list
    return info_list
    return score_list
    return comment_list

In [24]:
movie_list = list(movie())

# 保存信息

In [27]:
dataframe = pd.DataFrame({
    '名字':movie_list[0],
    '基本信息':movie_list[1],
    '分数':movie_list[2],
    '评价人数':movie_list[3]
})

In [28]:
dataframe.to_csv("09_豆瓣.csv",index=False,encoding="utf-8")